In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)   

# Scraping + Basic Data Prep


* concat all 26 websites in 1 dataframe
* adjust dataframe
* change datatype

In [2]:
main_url = 'https://sgulyano.github.io/eval/'

html_url = ['02226005.html',
            '08846113.html',
            '19689783.html',
            '24363682.html',
            '24674969.html',
            '26742703.html',
            '33472214.html',
            '33561138.html',
            '33565025.html',
            '41557226.html',
            '43951333.html',
            '44868136.html',
            '45435188.html',
            '48033334.html',
            '51435303.html',
            '53231654.html',
            '60189297.html',
            '61116032.html',
            '66887754.html',
            '68346670.html',
            '71728345.html',
            '79550445.html',
            '83255004.html',
            '94026030.html',
            '95860997.html',
            '96905621.html']
eval_url = [main_url + url for url in html_url]

def get_table_data(item):
    data = []
    table_body = item.find("tbody")
    rows = table_body.find_all("tr")
    for row in rows :
        cols = row.find_all(["td","th"])
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return data
    
dfs = []
for i in range(26):
    dfs.append(f"df{i}")


for j in range(26):
    page  =j
    r = requests.get(eval_url[page])
    soup = BeautifulSoup(r.content,"html.parser")
    tables = soup.find_all("table")
    head = pd.DataFrame(get_table_data(tables[0]))
    section = head.iloc[1,1]

# Adjust dataframe
    df = pd.DataFrame(get_table_data(tables[2]))
    df.dropna(how="all",inplace=True)
    df.columns = df.iloc[1,:]
    rows = [0,1]
    df.drop(rows,axis=0,inplace=True)
    df.set_index("ที่",inplace=True)
    df.drop(df[df.index.str.contains("รวม")].index,inplace=True)
    df.dropna(how= "any" , inplace= True)


# change all column "ร้อยละ" name
    cols = []
    count = 5
    for column in df.columns:
        if column == 'ร้อยละ':
            cols.append(f'{count}(%)')
            count-=1
            continue
        cols.append(column)
    df.columns = cols

    for i in range(1,15):
        df.iloc[:,i] = pd.to_numeric(df.iloc[:,i])

# add  detail to data frame

    df["degree"] = head.iloc[3,1]
    df["semester"] = head.iloc[2,1]  
    df["subject"] = head.iloc[0,1]
    df["section"] = head.iloc[1,1]
    df["นักศึกษาที่จดทะเบียน"] = head.iloc[4,1]
    df["นักศึกษาที่ประเมิน"] = head.iloc[5,1]


    dfs[j] = df
    
# concat all websites
result = pd.concat([dfs[i] for i in range(26)])

# separate year & semester 
result["year"] = ""
result["sem"] = ""

count = 0
for i in result["semester"]:
    yr_splt = i.split("/")
    result["year"][count] = yr_splt[1]
    result["sem"][count] = yr_splt[0]
    count += 1

# display
result



,ข้อคำถาม,5,5(%),4,4(%),3,3(%),2,2(%),1,...,X-Bar,SD,degree,semester,subject,section,นักศึกษาที่จดทะเบียน,นักศึกษาที่ประเมิน,year,sem
ที่,,,,,,,,,,,,,,,,,,,,,
1,สอนอย่างเป็นระบบ,4,66.67,1,16.67,0,0.00,1,16.67,0,...,4.33,1.21,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1
2,สอนให้คิด วิเคราะห์ วิจารณ์,4,66.67,0,0.00,1,16.67,1,16.67,0,...,4.17,1.33,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1
3,กระตุ้นบรรยากาศการเรียนการสอน มีวิธีสอนที่น่าส...,2,33.33,1,16.67,1,16.67,1,16.67,1,...,3.33,1.63,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1
4,เปิดโอกาสให้แสดงความคิดเห็นและแลกเปลี่ยนประสบก...,3,50.00,2,33.33,0,0.00,0,0.00,1,...,4.00,1.55,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1
5,มีการให้นำเสนองาน/มีการวัดผลโดยวิธีอื่นๆที่สะท...,3,50.00,2,33.33,1,16.67,0,0.00,0,...,4.33,0.82,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ผู้สอนแทรกเนื้อหาเกี่ยวกับคุณธรรมจริยธรรมในการ...,15,51.72,9,31.03,3,10.34,2,6.90,0,...,4.28,0.92,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),2563,1
1,โสตทัศนูปกรณ์ช่วยสอนในห้องเรียน เช่น ไมโครโฟน ...,14,48.28,8,27.59,5,17.24,1,3.45,1,...,4.14,1.06,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),2563,1
2,สภาพของห้องเรียนหรือห้องปฏิบัติการ เช่น แสง อุ...,13,44.83,11,37.93,4,13.79,0,0.00,1,...,4.21,0.94,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),2563,1


# Situation Separate

* If question contain "covid-19" then situation = "covid"
* then else situaion is "normal"

In [12]:
result["class"] = result["semester"] + "|" + result["subject"] + "|" + result["section"]

covid = result[result["ข้อคำถาม"].str.contains("COVID-19")]
covid["situation"] = "covid"

covid = covid.loc[:, df.columns.intersection(['class','situation'])]
covid.drop_duplicates(inplace= True)
df = pd.merge(result,covid,on="class" , how="outer")
df["situation"].fillna("normal" , inplace=True)
df.rename(columns={"X-Bar" : "x_bar"},inplace= True)

df

,ข้อคำถาม,5,5(%),4,4(%),3,3(%),2,2(%),1,...,degree,semester,subject,section,นักศึกษาที่จดทะเบียน,นักศึกษาที่ประเมิน,year,sem,class,situation
0,สอนอย่างเป็นระบบ,4,66.67,1,16.67,0,0.00,1,16.67,0,...,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1,1/2562|FGL497|231863,normal
1,สอนให้คิด วิเคราะห์ วิจารณ์,4,66.67,0,0.00,1,16.67,1,16.67,0,...,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1,1/2562|FGL497|231863,normal
2,กระตุ้นบรรยากาศการเรียนการสอน มีวิธีสอนที่น่าส...,2,33.33,1,16.67,1,16.67,1,16.67,1,...,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1,1/2562|FGL497|231863,normal
3,เปิดโอกาสให้แสดงความคิดเห็นและแลกเปลี่ยนประสบก...,3,50.00,2,33.33,0,0.00,0,0.00,1,...,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1,1/2562|FGL497|231863,normal
4,มีการให้นำเสนองาน/มีการวัดผลโดยวิธีอื่นๆที่สะท...,3,50.00,2,33.33,1,16.67,0,0.00,0,...,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),2562,1,1/2562|FGL497|231863,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,ผู้สอนแทรกเนื้อหาเกี่ยวกับคุณธรรมจริยธรรมในการ...,15,51.72,9,31.03,3,10.34,2,6.90,0,...,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),2563,1,1/2563|NYE553|231863,normal
320,โสตทัศนูปกรณ์ช่วยสอนในห้องเรียน เช่น ไมโครโฟน ...,14,48.28,8,27.59,5,17.24,1,3.45,1,...,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),2563,1,1/2563|NYE553|231863,normal
321,สภาพของห้องเรียนหรือห้องปฏิบัติการ เช่น แสง อุ...,13,44.83,11,37.93,4,13.79,0,0.00,1,...,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),2563,1,1/2563|NYE553|231863,normal
322,จำนวนอุปกรณ์ในการเรียนการสอนเพียงพอ และเหมาะสม...,15,51.72,6,20.69,5,17.24,0,0.00,3,...,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),2563,1,1/2563|NYE553|231863,normal
